In [10]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
%matplotlib inline

In [11]:
air_trajs = pd.read_csv('air_trajs.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

In [12]:
end_points = np.array([trip[-1] for trip in air_trajs['POLYLINE']])
start_points = np.array([trip[0] for trip in air_trajs['POLYLINE']])

In [37]:
def index_points_in_circe(center, radius, points):
    indexes = []
    for index, point in points.iteritems():
        if haversine(center, point) <= radius:
            indexes.append(index)
            
    return indexes

In [41]:
# we need formaula to compute distance between two points

#Since the distance is relatively small, you can use the equirectangular distance approximation
def haversine(point1, point2):
    # radius of the earth in km
    R = 6371 
    # get lat and lon
    lon1, lat1 = point1[0], point1[1]
    lon2, lat2 = point2[0], point2[1]
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # formula
    x = (lon2 - lon1) * math.cos( 0.5*(lat2+lat1) )
    y = lat2 - lat1
    d = R * math.sqrt( x*x + y*y )
    return d



In [153]:
start_hash_points = start_points.copy()
for index, point in start_points.iteritems():
    start_hash_points[index] = Geohash.encode(point[1], point[0])

start_hash_points = np.array(start_hash_points)

In [154]:
geo_index = GeoGridIndex()
for point in start_points:
    #print point
    geo_index.add_point(GeoPoint(point[1], point[0]))

In [156]:
center_point = GeoPoint(start_points[0][1], start_points[0][0])
indexes = []
for point, distance in geo_index.get_nearest_points(center_point, 0.02, 'km'):
    hash_point = Geohash.encode(point.latitude, point.longitude)
    p_indexes = list(np.where(start_hash_points == hash_point)[0])
    indexes.extend(p_indexes)
    
    #print("We found {0} in {1} km".format(point, distance))

In [157]:
print len(indexes)
print len(list(set(indexes)))

165
145


In [49]:
start_points = air_trajs['POLYLINE'].copy()
for index, row in start_points.iteritems():
    start_points[index] = row[0]

In [160]:
first = []
second = []
third = []
import time

In [162]:
time_start = time.clock()

for point in start_points:
    center_point = GeoPoint(point[1], point[0])
    first.append(geo_index.get_nearest_points(center_point, 0.02, 'km'))
    
time_elapsed = (time.clock() - time_start)
print time_elapsed

0.0247949999999


In [163]:
time_start = time.clock()

for point in start_points:
    second.append(index_points_in_circe(point, 0.02, start_points))
    
time_elapsed = (time.clock() - time_start)
print time_elapsed

47.189968


In [161]:
time_start = time.clock()

for point in start_points:
    center_point = GeoPoint(point[1], point[0])
    indexes = []
    for point, distance in geo_index.get_nearest_points(center_point, 0.02, 'km'):
        hash_point = Geohash.encode(point.latitude, point.longitude)
        p_indexes = list(np.where(start_hash_points == hash_point)[0])
        indexes.extend(p_indexes)
        
    indexes = list(set(indexes))
    third.append(indexes)
    
    
time_elapsed = (time.clock() - time_start)
print time_elapsed

212.680808


In [164]:
counter = 0
time_start = time.clock()

for point in start_points:
    center_point = GeoPoint(point[1], point[0])
    indexes = []
    for point, distance in geo_index.get_nearest_points(center_point, 0.02, 'km'):
        counter =+ 1
    
time_elapsed = (time.clock() - time_start)
print time_elapsed

128.657799
